<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.3
# *The Google BigQuery UI and API*

## Introduction

The Google BigQuery UI provides access to Google's extensive collection of public data sets via an SQL-based query engine.

The BigQuery API provides programmatic access to the data sets.

We can use the UI to discover interesting data before writing Python code to access it. Then we can reproduce it in an API request so as to aggregate large amounts of data on Google's infrastructure before pulling the results into our application.

## BigQuery Web UI

Work through the Quickstart at https://cloud.google.com/bigquery/docs/quickstarts/quickstart-web-ui.

You will need to set up a Google Cloud Platform account if you don't already have one. (This should not cost anything during the trial period unless you perform a large amount of querying. Afterwards, costs are based on actual resource usage, but most offerings have a free tier.)

## BigQuery API

You should already have the Google Cloud Client Library for Python installed (https://cloud.google.com/python/setup).

- Open Google Cloud Console (https://console.cloud.google.com/home/) and select to create a project.


- Under "Getting Started", select "Enable APIs and get credentials such as keys".

- In the API table, make sure the BigQuery API is enabled. Page back.

### Authentication

Go to https://cloud.google.com/iam/docs/creating-managing-service-account-keys and follow the instructions under "Create a service account key" to create a service account.

- Fill out the form, giving the account an appropriate name, and choose "Project Owner" for Account Type.

- Click "Create".

- The keys will get saved to your computer.

- Note the location and copy the file path (of the json file) to somewhere safe, for future reference.

- See here for more information:

https://cloud.google.com/iam/docs/understanding-service-accounts?&_ga=2.173177830.-495703703.1532572448#managing_service_account_keys

This is supposed to get implicit key retrieval working:

- Windows:
    `set GOOGLE_APPLICATION_CREDENTIALS=[PATH]`
    
- Linux, MacOS:
    `export GOOGLE_APPLICATION_CREDENTIALS=[PATH]`
    
where `[PATH]` is the full file path of your json key file.

### Using the Python API

Google provides Python libraries for wrapping the Google APIs. For conda users, these are available on the "conda-forge" channel. (There are other Python libraries for Google APIs, possibly adding novel features or ease of use, but Google's is presumably the most current.)

(Installing the "google-cloud-storage" and "google-cloud-bigquery" libraries should cover all the dependencies for this lab.)

In [ ]:
from google.cloud import bigquery
from google.cloud.bigquery import Dataset
from google.cloud import storage

Invoke a method of the `.Client` object that takes the path to your key files as a string argument:

In [1]:
key_path = 'auth_gc_bigquery.json'          #: This file should contain your key path

This should not throw an error if key retrieval / assignment worked:

In [ ]:
storage_client = storage.Client.from_service_account_json(key_path)

*Nb. The `storage` object was used in the above example, but there are other objects of interest that have polymorphic `Client` members that are used similarly, such as `bigquery`, which is used below.*

Next, execute this:

In [ ]:
client = bigquery.Client.from_service_account_json(key_path)

This client is associated with the default project (which was set or defaulted in the BigQuery UI):

In [ ]:
client.project

'myreallybigquery'

A BigQuery project contains datasets. Datasets contain tables. To get at the data in a table we need to create a reference that covers this hierarchy; in the `bigquery` library this looks like `project.dataset.table`.  

(Nb. Queries can be performed on projects and datasets, but most queries are performed on tables.)

To explore the public datasets we will start by reassigning our `client` variable using optional `project` parameter (set to `bigquery-public-data`):

In [ ]:
#project = 'bigquery-public-data'
client = bigquery.Client.from_service_account_json(key_path, project = 'bigquery-public-data')
print(client.project)

bigquery-public-data


Here is how to get a list of the datasets in the current project:

In [ ]:
datasets = list(client.list_datasets())
print(datasets)

[<google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341AE48278>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C4485C0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C448E10>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C448E48>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C4208D0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C420898>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C473DD8>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C4736A0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C473D68>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C473C50>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C473D30>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000002341C473BA8>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000

That wasn't helpful. We need to go deeper into the object structure to get at something meaningful. Actually, the `dataset_id` member contains the name attribute of a `dataset` object; write some code to print that name for each member of the list that was created above:

In [ ]:
#?

The google API objects in the `bigquery` library have their own overloads of the format() function that make them easier to read. Below is a function that exploits the `format` method of `project` and `dataset_id`, providing an easy way to list datasets:

In [ ]:
# function for listing datasets in a project:
def printDatasetList(client):
    project = client.project    #: only one project can be associated with a client instance
    datasets = list(client.list_datasets())
    if datasets:
        print('Datasets in project {}:'.format(project))
        for dataset in datasets:
            print('\t{}'.format(dataset.dataset_id))
        found = True
    else:
        print('{} project does not contain any datasets.'.format(project))
        found = False
    return found

In [ ]:
# list datasets in the default project:
flag = printDatasetList(client)  #: assigning to `flag` suppresses printing the return value (normally `True`)

Datasets in project bigquery-public-data:
	austin_311
	austin_bikeshare
	austin_crime
	austin_incidents
	austin_waste
	baseball
	bitcoin_blockchain
	bls
	census_bureau_construction
	census_bureau_international
	census_bureau_usa
	chicago_crime
	chicago_taxi_trips
	cloud_storage_geo_index
	cms_codes
	cms_medicare
	eclipse_megamovie
	epa_historical_air_quality
	ethereum_blockchain
	fda_drug
	fda_food
	fec
	genomics_cannabis
	ghcn_d
	ghcn_m
	github_repos
	google_analytics_sample
	google_political_ads
	hacker_news
	human_variant_annotation
	irs_990
	libraries_io
	london_bicycles
	london_crime
	london_fire_brigade
	medicare
	moon_phases
	ncaa_basketball
	new_york
	new_york_311
	new_york_citibike
	new_york_mv_collisions
	new_york_taxi_trips
	new_york_trees
	nhtsa_traffic_fatalities
	nlm_rxnorm
	noaa_goes16
	noaa_gsod
	noaa_hurricanes
	noaa_icoads
	noaa_significant_earthquakes
	noaa_spc
	nppes
	open_images
	openaq
	samples
	san_francisco
	san_francisco_311
	san_francisco_film_locations
	san_f

This list should correspond to what is shown here https://bigquery.cloud.google.com/publicdatasets under the **bigquery-public-data** item.

Here is how to create a dataset reference object by assigning a project and a dataset name:

In [ ]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id)

If our current project was something other than `bigquery-public-data`, we could still create this reference by specifying the project that contains the dataset:

In [ ]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id, project = 'bigquery-public-data')

How can we get the path of the dataset?

In [ ]:
#ANSWER:
dataset_ref.path

'/projects/bigquery-public-data/datasets/samples'

Explore more of this object's members:

*(HINT: Jupyter Notebooks does not support code completion, but Spyder and other Python IDEs do. If you copy all the above code to a Python file within the IDE, you can type `dataset_ref.` in a new line, then hit the [Tab] key to see the available members for the object.)*

In [ ]:
#?
dataset_ref.table(table_id)


SyntaxError: invalid syntax (<ipython-input-16-6e2098437bbd>, line 2)

Here is a function for listing the tables in a dataset:

In [ ]:
# function for listing tables in a dataset:
def printTableList(client, dataset_id):
    project = client.project
    dataset_ref = client.dataset(dataset_id, project = project)
    tables = list(client.list_tables(dataset_ref))
    if tables:
        print('Tables in dataset {}:'.format(dataset_id))
        for table in tables:
            print('\t{}'.format(table.table_id))
        found = True
    else:
        print('{} dataset does not contain any tables.'.format(dataset_id))
        found = False
    return found

Use this function to list the tables in the current dataset:

In [ ]:
#ANSWER
printTableList(client, dataset_id)

Tables in dataset samples:
	github_nested
	github_timeline
	gsod
	model
	natality
	shakespeare
	trigrams
	wikipedia


True

To create a reference to a table within the dataset, we use the `table_id` attribute:

In [ ]:
table_id = 'shakespeare'
table_ref = dataset_ref.table(table_id)

Check the name of the table that `table_ref` now points to:

In [ ]:
#ANSWER
table_ref.table_id

'shakespeare'

To access the data in the table itself, we use the `get_table()` method:

In [ ]:
table = client.get_table(table_ref)  # API Request

NOTE: The contents of the table are not actually in our memory after this call! We are working with a Big Data platform, now, and we could easily end up pulling GBs or TBs of data by accident.

To minimise data bandwidth, memory consumption, and processing time, Big Data platforms employ ***lazy evaluation***. This means that no computation or data transfer actually takes place until we *realise* (use) the data. Even if we execute subsequent code that performs calculations on the data, no data flow or computation actually occurs until we request output (e.g. by executing a print to stdout or writing to a file).

What kind of object is returned by `client.get_table`?

In [ ]:
#ANSWER:
type(table)

google.cloud.bigquery.table.Table

How can we view the design of the table (column names and types? The name of the object attribute we need is the same term we learned in the module on databases:

In [ ]:
#ANSWER
print(table.schema)

[SchemaField('word', 'string', 'REQUIRED', 'A single unique word (where whitespace is the delimiter) extracted from a corpus.', ()), SchemaField('word_count', 'integer', 'REQUIRED', 'The number of times this word appears in this corpus.', ()), SchemaField('corpus', 'string', 'REQUIRED', 'The work from which this word was extracted.', ()), SchemaField('corpus_date', 'integer', 'REQUIRED', 'The year in which this corpus was published.', ())]


Again, this is messy. If we wanted to refer to the column names and types in code, we might use something like this (which we could then parse into a dict):

In [ ]:
result = ["{0} {1}".format(schema.name,schema.field_type) for schema in table.schema]
print(result)

['word STRING', 'word_count INTEGER', 'corpus STRING', 'corpus_date INTEGER']


But if we just want to print them, here is another neat function for that:

In [ ]:
# function to print a table schema:
def printTableSchema(aTable):
    schemas = list(aTable.schema)
    if schemas:
        print('Table schema for {}:'.format(aTable.table_id))
        for aSchema in schemas:
            print('\t{0} {1}'.format(aSchema.name, aSchema.field_type))
        found = True
    else:
        found = False
    return found

Use this function to print the table schema:

In [ ]:
#ANSWER:
printTableSchema(table)

Table schema for shakespeare:
	word STRING
	word_count INTEGER
	corpus STRING
	corpus_date INTEGER


True

Now that we know what the columns are, we can write queries. Actually, we construct a query job by assigning an SQL statement to a method of the `client` object:

In [ ]:
sql = "SELECT COUNT(1) FROM bigquery-public-data.samples.shakespeare"
query_job = client.query(sql)

Forbidden: 403 POST https://www.googleapis.com/bigquery/v2/projects/bigquery-public-data/jobs: Access Denied: Project bigquery-public-data: The user bigquery-api-owner@myreallybigquery.iam.gserviceaccount.com does not have bigquery.jobs.create permission in project bigquery-public-data.

Why does this throw an error?

ANSWER:

We don't have permission to create queries inside the `bigquery-public-data` project.

So, what can we do?

In [ ]:
#ANSWER
client = bigquery.Client.from_service_account_json(key_path, project = 'myreallybigquery') #<<< your BigQuery project ID here!
query_job = client.query(sql)

If that worked, show what query_job is:

In [ ]:
#ANSWER
type(query_job)

google.cloud.bigquery.job.QueryJob

Once again, due to lazy execution, no actual execution occurs until we request output:

In [ ]:
for row in query_job:  # API request - fetches results
    print(row)

And, again, we need to manipulate this to make it neat. Each member of the rowset is a list and we only want to extract the value, which is in the first member:

In [ ]:
print(row[0])

164656


So, we now know that this table has 164,656 rows. (We would not want to print it!)

A better coding practice is to write SQL statements that assign names (aliases) to derived values, so we don't forget what the resulting rowset contains. Rewrite the above SQL statement so that the value returned is aliased a "num_rows", and assign the QueryJob as above:

In [ ]:
#ANSWER
sql = "SELECT COUNT(1) AS num_rows FROM bigquery-public-data.samples.shakespeare"
query_job = client.query(sql)

Now we could use Python's `assert` statement to build a test into the first code block that operates on the rowset:

In [ ]:
for row in query_job:  # API request - fetches results
    # Row values can be accessed by field name or index:
    assert row[0] == row.num_rows == row['num_rows']  #: for debugging bad sql
    print(row.num_rows)

164656


The above code checks that the name attribute of the value in `row[0]` is what we expected (i.e. "num_rows"). Also, it shows that we can refer to a field in a row by its object member `num_rows` or by using the same notation we use for Python dictionaries, `['num_rows']`.

Write, execute, and print the results of a query that fetches 10 rows from the table, each containing the "word", "word_count", and "corpus" fields:

In [ ]:
#ANSWER
sql = "SELECT word, word_count, corpus FROM bigquery-public-data.samples.shakespeare LIMIT 10"
query_job = client.query(sql) #E: , location='US') #: OK if client.project = 'myreallybigquery'

# print these as above:
for row in query_job:  # API request - fetches results
    # Now have 3 fields to test (Nb. this approach may be overkill for non-production code):
    assert row[0] == row.word == row['word']
    assert row[1] == row.word_count == row['word_count']
    assert row[2] == row.corpus == row['corpus']
    print(row['word'], row['word_count'], row['corpus'])

LVII 1 sonnets
augurs 1 sonnets
dimm'd 1 sonnets
plagues 1 sonnets
treason 1 sonnets
surmise 1 sonnets
heed 1 sonnets
Unthrifty 1 sonnets
quality 1 sonnets
wherever 1 sonnets


(NOTE: Using `assert` religiously is good practice and will make debugging easier, but is probably overkill for non-production code.)

Whenever you catch yourself writing a swag of code to do something that seems rudimentary or low-level, there is a very good chance that you don't need to. A much easier way to handle the above requirement is to use the `to_dataframe` method of the QueryJob object:

In [ ]:
df = query_job.to_dataframe()
print(df)

        word  word_count   corpus
0       LVII           1  sonnets
1     augurs           1  sonnets
2     dimm'd           1  sonnets
3    plagues           1  sonnets
4    treason           1  sonnets
5    surmise           1  sonnets
6       heed           1  sonnets
7  Unthrifty           1  sonnets
8    quality           1  sonnets
9   wherever           1  sonnets


Although the above doesn't use `assert` (which you might still want to include in some test code), you will be able to tell at a glance if something is wrong with the contents of the DataFrame.

#### Final Notes

1. Here is a readable way to code long SQL statements:

In [ ]:
sql = """
    SELECT word, word_count, corpus
    FROM bigquery-public-data.samples.shakespeare
    LIMIT 10
    """

2. If you had an application that needed to modify the tables or datasets in the `bigquery-public-data` is project, you could copy them to our own project, where you would have the permissions to do as you please with the data (subject to Google's terms of use).

3. We aren't limited to the datasets that are already in BigQuery. We can upload tables from our computer, and we can pull data in from other online sources. We will cover these tasks in another module.

#### Next Steps

If you wish to pick up a few more skills you can go to https://cloud.google.com/bigquery/create-simple-app-api. (Note that we have already been through the preliminaries, so you can start at "Download the sample code".)

Alternatively, you can take a deeper dive into the API here: https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html.

## - END -



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



